# Sign Language Recognition Using LSTM DL Model

In [ ]:
!pip install tensorflow==2.5.0 tensorflow-gpu==2.5.0 opencv-python mediapipe sklearn matplotlib

## Installing and importing dependencies

In [ ]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## Marking the keypoints

In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def mediapipe_det(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image.flags.writeable = False
    res = model.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image, res

In [ ]:
def styled_landmarks(image, model):
    # Face
    mp_drawing.draw_landmarks(
        image, res.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
        mp_drawing.DrawingSpec(color=(80, 110, 10),
                               thickness=1,
                               circle_radius=1),
        mp_drawing.DrawingSpec(color=(80, 256, 121),
                               thickness=1,
                               circle_radius=1))
    # Pose
    mp_drawing.draw_landmarks(
        image, res.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80, 22, 10),
                               thickness=2,
                               circle_radius=4),
        mp_drawing.DrawingSpec(color=(80, 44, 121),
                               thickness=2,
                               circle_radius=2))
    # Left hand
    mp_drawing.draw_landmarks(
        image, res.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121, 22, 76),
                               thickness=2,
                               circle_radius=4),
        mp_drawing.DrawingSpec(color=(121, 44, 250),
                               thickness=2,
                               circle_radius=2))
    # Right hand
    mp_drawing.draw_landmarks(
        image, res.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245, 117, 66),
                               thickness=2,
                               circle_radius=4),
        mp_drawing.DrawingSpec(color=(245, 66, 230),
                               thickness=2,
                               circle_radius=2))

In [ ]:
cap = cv.VideoCapture(0)
# Setting the model
with mp_holistic.Holistic(min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read capture
        ret, frame = cap.read()

        # Detect
        image, res = mediapipe_det(frame, holistic)

        # Draw
        styled_landmarks(image, res)

        cv.imshow("Camera Feed", image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

## Extracting the values

In [ ]:
pose = []
for resV in res.pose_landmarks.landmark:
    test = np.array([resV.x, resV.y, resV.z, resV.visibility])
    pose.append(test)

In [ ]:
def extract(res):
    # 33 * 4 = 33 landmarks, 4 values each (includes resV.visibility)
    pose = np.array([[resV.x, resV.y, resV.z, resV.visibility]
                     for resV in res.pose_landmarks.landmark
                     ]).flatten() if res.pose_landmarks else np.zeros(132)
    # 468 * 3 = 468 landmarks, 3 values each
    face = np.array([[resV.x, resV.y, resV.z]
                     for resV in res.face_landmarks.landmark
                     ]).flatten() if res.face_landmarks else np.zeros(1404)
    lHand = np.array([[resV.x, resV.y, resV.z]
                      for resV in res.left_hand_landmarks.landmark
                      ]).flatten() if res.left_hand_landmarks else np.zeros(63)
    rHand = np.array([
        [resV.x, resV.y, resV.z] for resV in res.right_hand_landmarks.landmark
    ]).flatten() if res.right_hand_landmarks else np.zeros(63)
    # 21 * 3 = 21 landmarks, 3 values each
    return np.concatenate([pose, face, lHand, rHand])

## Collection file system

In [ ]:
DATA_PATH = os.path.join('MP_Data')
actions = np.array(['hello', "thanks", "i love you"])

# 30 seqences
no_seq = 30

# Every sequence 30 frames long
seq_length = 30

In [ ]:
for action in actions:
    for seq in range(no_seq):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(seq)))
        except:
            pass

## Collecting values from keypoints

In [ ]:
cap = cv.VideoCapture(0)
# Setting the model
with mp_holistic.Holistic(min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for seq in range(no_seq):
            for frame_num in range(seq_length):

                # Read capture
                ret, frame = cap.read()

                # Detect
                image, res = mediapipe_det(frame, holistic)

                # Draw
                styled_landmarks(image, res)

                # Wait logic
                if frame_num == 0:
                    cv.putText(image, 'Starting collection', (120, 200),
                               cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4,
                               cv.LINE_AA)
                    cv.putText(
                        image,
                        'Collecting frames for {} Video Number {}'.format(
                            action, seq), (15, 12), cv.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 0, 255), 1, cv.LINE_AA)
                    # Show to screen
                    cv.imshow('OpenCV Feed', image)
                    cv.waitKey(500)
                else:
                    cv.putText(
                        image,
                        'Collecting frames for: {}. Seq. Number: {}.'.format(
                            action, seq), (15, 12), cv.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 0, 255), 1, cv.LINE_AA)
                    # Show to screen
                    cv.imshow('OpenCV Feed', image)

                # Export
                keypoints = extract(res)
                np_path = os.path.join(DATA_PATH, action, str(seq),
                                       str(frame_num))
                np.save(np_path, keypoints)

                if cv.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv.destroyAllWindows()

## Preprocess and label

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label: num for num, label in enumerate(actions)}

In [ ]:
label_map